In [1]:
import open3d as o3d
from numba.core.errors import NumbaDeprecationWarning,NumbaPendingDeprecationWarning, NumbaWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaWarning)
import sys
sys.path.append('/kaggle/code/ConeDetectionPointpillars')

from second.data.CustomNuscDataset import * #to register dataset
from models import * #to register model
import torch
from second.utils.log_tool import SimpleModelLog
from second.builder import target_assigner_builder, voxel_builder
from second.protos import pipeline_pb2
from second.pytorch.builder import (box_coder_builder, input_reader_builder,
                                    lr_scheduler_builder, optimizer_builder,
                                    second_builder)
from second.pytorch.core import box_torch_ops
import torchplus
import re
import time
from pathlib import Path
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from google.protobuf import text_format
# from lyft_dataset_sdk.utils.geometry_utils import *
from lyft_dataset_sdk.lyftdataset import Quaternion
from apex import amp
# from lyft_dataset_sdk.utils.data_classes import Box
from nuscenes.utils.geometry_utils import *
from nuscenes.utils.data_classes import Box
from second.utils.progress_bar import ProgressBar
from tqdm import tqdm, tqdm_notebook

#commented second.core.non_max_suppression, nms_cpu, __init__.py, 
# pytorch.core.box_torch_ops line 524

tensor([[0.7674, 0.1264]])
tensor([[-0.0363,  0.7567]], device='cuda:0')
tensor([[0.7674, 0.1264]])
False
tensor([[0.7674, 0.1264]], device='cuda:0')
True
True


In [2]:
def _second_det_to_nusc_box(detection):
    from nuscenes.utils.data_classes import Box
    import pyquaternion
    box3d = detection["box3d_lidar"].detach().cpu().numpy()
    scores = detection["scores"].detach().cpu().numpy()
    labels = detection["label_preds"].detach().cpu().numpy()
    print(labels)
    box3d[:, 6] = -box3d[:, 6] - np.pi / 2
    box_list = []
    for i in range(box3d.shape[0]):
        quat = pyquaternion.Quaternion(axis=[0, 0, 1], radians=box3d[i, 6])
        velocity = (np.nan, np.nan, np.nan)
        if box3d.shape[1] == 9:
            velocity = (*box3d[i, 7:9], 0.0)
            # velo_val = np.linalg.norm(box3d[i, 7:9])
            # velo_ori = box3d[i, 6]
            # velocity = (velo_val * np.cos(velo_ori), velo_val * np.sin(velo_ori), 0.0)
        box = Box(
            box3d[i, :3],
            box3d[i, 3:6],
            quat,
            label=labels[i],
            score=scores[i],
            velocity=velocity)
        box_list.append(box)
    return box_list
def build_network(model_cfg, measure_time=False):
    voxel_generator = voxel_builder.build(model_cfg.voxel_generator)
    bv_range = voxel_generator.point_cloud_range[[0, 1, 3, 4]]
    box_coder = box_coder_builder.build(model_cfg.box_coder)
    target_assigner_cfg = model_cfg.target_assigner
    target_assigner = target_assigner_builder.build(target_assigner_cfg,
                                                    bv_range, box_coder)
    box_coder.custom_ndim = target_assigner._anchor_generators[0].custom_ndim
    net = second_builder.build(
        model_cfg, voxel_generator, target_assigner, measure_time=measure_time)
    return net


def merge_second_batch_multigpu(batch_list):
    if isinstance(batch_list[0], list):
        batch_list_c = []
        for example in batch_list:
            batch_list_c += example
        batch_list = batch_list_c

    example_merged = defaultdict(list)
    for example in batch_list:
        for k, v in example.items():
            example_merged[k].append(v)
    ret = {}
    for key, elems in example_merged.items():
        if key == 'metadata':
            ret[key] = elems
        elif key == "calib":
            ret[key] = {}
            for elem in elems:
                for k1, v1 in elem.items():
                    if k1 not in ret[key]:
                        ret[key][k1] = [v1]
                    else:
                        ret[key][k1].append(v1)
            for k1, v1 in ret[key].items():
                ret[key][k1] = np.stack(v1, axis=0)
        elif key == 'coordinates':
            coors = []
            for i, coor in enumerate(elems):
                coor_pad = np.pad(
                    coor, ((0, 0), (1, 0)), mode='constant', constant_values=i)
                coors.append(coor_pad)
            ret[key] = np.stack(coors, axis=0)
        elif key in ['gt_names', 'gt_classes', 'gt_boxes']:
            continue
        else:
            ret[key] = np.stack(elems, axis=0)

    return ret

def buildBBox(points,color = [1,0,0]):
    #print("Let's draw a cubic using o3d.geometry.LineSet")
    # points = [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1], [1, 0, 1],
    #           [0, 1, 1], [1, 1, 1]] x0y0z0, x0y0z1, x0y1z0, x0y1z1, x1y0z0, x1y0z1, x1y1z0, x1y1z1

    points = points[[0,4,3,7,1,5,2,6],:]
    lines = [[0, 1], [0, 2], [1, 3], [2, 3], [4, 5], [4, 6], [5, 7], [6, 7],
             [0, 4], [1, 5], [2, 6], [3, 7]]
    colors = [color for i in range(len(lines))]
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)


    return  line_set
def merge_second_batch(batch_list):
    if isinstance(batch_list[0], list):
        batch_list_c = []
        for example in batch_list:
            batch_list_c += example
        batch_list = batch_list_c

    example_merged = defaultdict(list)
    for example in batch_list:
        for k, v in example.items():
            example_merged[k].append(v)
    ret = {}
    for key, elems in example_merged.items():
        if key in [
            'voxels', 'num_points', 'num_gt', 'voxel_labels', 'gt_names', 'gt_classes', 'gt_boxes'
        ]:
            ret[key] = np.concatenate(elems, axis=0)
        elif key == 'metadata':
            ret[key] = elems
        elif key == "calib":
            ret[key] = {}
            for elem in elems:
                for k1, v1 in elem.items():
                    if k1 not in ret[key]:
                        ret[key][k1] = [v1]
                    else:
                        ret[key][k1].append(v1)
            for k1, v1 in ret[key].items():
                ret[key][k1] = np.stack(v1, axis=0)
        elif key == 'coordinates':
            coors = []
            for i, coor in enumerate(elems):
                coor_pad = np.pad(
                    coor, ((0, 0), (1, 0)), mode='constant', constant_values=i)
                coors.append(coor_pad)
            ret[key] = np.concatenate(coors, axis=0)
        elif key == 'metrics':
            ret[key] = elems
        else:
            ret[key] = np.stack(elems, axis=0)
    return ret


def _worker_init_fn(worker_id):
    time_seed = np.array(time.time(), dtype=np.int32)
    np.random.seed(time_seed + worker_id)


def example_convert_to_torch(example, dtype=torch.float32,
                             device=None) -> dict:
    device = device or torch.device("cuda:0")
    example_torch = {}
    float_names = [
        "voxels", "anchors", "reg_targets", "reg_weights", "bev_map", "importance"
    ]
    for k, v in example.items():
        if k in ['gt_names', 'gt_classes', 'gt_boxes', 'points']:
            example_torch[k] = example[k]
            continue

        if k in float_names:
            # slow when directly provide fp32 data with dtype=torch.half
            example_torch[k] = torch.tensor(
                v, dtype=torch.float32, device=device).to(dtype)
        elif k in ["coordinates", "labels", "num_points"]:
            example_torch[k] = torch.tensor(
                v, dtype=torch.int32, device=device)
        elif k in ["anchors_mask"]:
            example_torch[k] = torch.tensor(
                v, dtype=torch.uint8, device=device)
        elif k == "calib":
            calib = {}
            for k1, v1 in v.items():
                calib[k1] = torch.tensor(
                    v1, dtype=dtype, device=device).to(dtype)
            example_torch[k] = calib
        elif k == "num_voxels":
            example_torch[k] = torch.tensor(v)
        else:
            example_torch[k] = v
    return example_torch


def time_to_str(t, mode='min'):
    if mode == 'min':
        t = int(t) / 60
        hr = t // 60
        min = t % 60
        return '%2d hr %02d m' % (hr, min)

    elif mode == 'sec':
        t = int(t)
        min = t // 60
        sec = t % 60
        return '%2d min %02d sec' % (min, sec)


    else:
        raise NotImplementedError


In [3]:
config_path = '/kaggle/code/ConeDetectionPointpillars/customNuscenes/configs/cones_pp_full_aug.config'
# config_path = '/kaggle/code/ConeDetectionPointpillars/customNuscenes/configs/cones_pp_initial_v2.config'
# config_path = '/kaggle/code/ConeDetectionPointpillars/customNuscenes/configs/cones_pp_initialak.config'
model_dir = f'/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug'
result_path = None
create_folder = False
display_step = 100
# pretrained_path="/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_9:53/voxelnet-5850.tckpt"
pretrained_path = '/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug/2021-2-13_2:56/voxelnet-32000.tckpt'
multi_gpu = False
measure_time = False
resume = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = str(Path(model_dir).resolve())
cur_time = time.localtime(time.time())
cur_time = f'{cur_time.tm_year}-{cur_time.tm_mon}-{cur_time.tm_mday}_{cur_time.tm_hour}:{cur_time.tm_min}'
if create_folder:
    if Path(model_dir).exists():
        model_dir = torchplus.train.create_folder(model_dir)
model_dir = Path(model_dir)

if not resume and model_dir.exists():
    raise ValueError("model dir exists and you don't specify resume")

model_dir.mkdir(parents=True, exist_ok=True)
if result_path is None:
    result_path = model_dir / 'results'/ cur_time
config_file_bkp = "pipeline.config"

if isinstance(config_path, str):
    config = pipeline_pb2.TrainEvalPipelineConfig()
    with open(config_path, "r") as f:
        proto_str = f.read()
        text_format.Merge(proto_str, config)
else:
    config = config_path
    proto_str = text_format.MessageToString(config, indent=2)

with (model_dir / config_file_bkp).open('w') as f:
    f.write(proto_str)
# Read config file
input_cfg = config.train_input_reader
eval_input_cfg = config.eval_input_reader
model_cfg = config.model.second  # model's config
train_cfg = config.train_config  # training config


In [4]:
# Build neural network
net = build_network(model_cfg, measure_time).to(device)

target_assigner = net.target_assigner
voxel_generator = net.voxel_generator
print("num parameter: ", len(list(net.parameters())))
torchplus.train.try_restore_latest_checkpoints(model_dir, [net])

num parameter:  64


In [5]:
if pretrained_path is not None:
    print('warning pretrain is loaded after restore, careful with resume')
    model_dict = net.state_dict()
    pretrained_dict = torch.load(pretrained_path)

    new_pretrained_dict = {}
    for k, v in pretrained_dict.items():
        if k in model_dict and v.shape == model_dict[k].shape:
            new_pretrained_dict[k] = v
    print("Load pretrained parameters: ")
    for k, v in new_pretrained_dict.items():
        print(k, v.shape)
    model_dict.update(new_pretrained_dict)
    net.load_state_dict(model_dict)
    net.clear_global_step()
    net.clear_metrics()
if multi_gpu:
    net_parallel = torch.nn.DataParallel(net)
else:
    net_parallel = net
optimizer_cfg = train_cfg.optimizer
loss_scale = train_cfg.loss_scale_factor
fastai_optimizer = optimizer_builder.build(
    optimizer_cfg,
    net,
    mixed=False,
    loss_scale=loss_scale)
if loss_scale < 0:
    loss_scale = "dynamic"
if train_cfg.enable_mixed_precision:
    max_num_voxels = input_cfg.preprocess.max_number_of_voxels * input_cfg.batch_size
    print("max_num_voxels: %d" % (max_num_voxels))

    net, amp_optimizer = amp.initialize(net, fastai_optimizer,
                                        opt_level="O1",
                                        keep_batchnorm_fp32=None,
                                        loss_scale=loss_scale)
    net.metrics_to_float()
else:
    amp_optimizer = fastai_optimizer
torchplus.train.try_restore_latest_checkpoints(model_dir, [fastai_optimizer])
lr_scheduler = lr_scheduler_builder.build(optimizer_cfg, amp_optimizer, train_cfg.steps)

if train_cfg.enable_mixed_precision:
    float_dtype = torch.float16
else:
    float_dtype = torch.float32

if multi_gpu:
    num_gpu = torch.cuda.device_count()
    print(f"MULTI_GPU: use {num_gpu} gpus")
    collate_fn = merge_second_batch_multigpu
else:
    collate_fn = merge_second_batch
    num_gpu = 1

warning pretrain is loaded after restore, careful with resume
Load pretrained parameters: 
global_step torch.Size([1])
voxel_feature_extractor.pfn_layers.0.linear.weight torch.Size([64, 10])
voxel_feature_extractor.pfn_layers.0.norm.weight torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.bias torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.running_mean torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.running_var torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.num_batches_tracked torch.Size([])
rpn.blocks.0.1.weight torch.Size([64, 64, 3, 3])
rpn.blocks.0.2.weight torch.Size([64])
rpn.blocks.0.2.bias torch.Size([64])
rpn.blocks.0.2.running_mean torch.Size([64])
rpn.blocks.0.2.running_var torch.Size([64])
rpn.blocks.0.2.num_batches_tracked torch.Size([])
rpn.blocks.0.4.weight torch.Size([64, 64, 3, 3])
rpn.blocks.0.5.weight torch.Size([64])
rpn.blocks.0.5.bias torch.Size([64])
rpn.blocks.0.5.running_mean torch.Size([64])
rpn.blocks.0.5.running_va

In [7]:
dataset = input_reader_builder.build(
    input_cfg,
    model_cfg,
    training=True,
    voxel_generator=voxel_generator,
    target_assigner=target_assigner,
    multi_gpu=multi_gpu
)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=input_cfg.batch_size * num_gpu,
    shuffle=True,
    num_workers=input_cfg.preprocess.num_workers * num_gpu,
    pin_memory=False,
    collate_fn=collate_fn,
    worker_init_fn=_worker_init_fn,
    drop_last=not multi_gpu
)


feature_map_size [1, 100, 150]
Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 30.605 seconds.
Reverse indexing ...
Done reverse indexing in 5.4 seconds.


In [8]:
dataset[1]

INFO - 2021-02-15 21:12:56,483 - transforms - finding looplift candidates


{'voxels': array([[[-1.36781321, -2.38647573,  0.100171  , -0.44117647,
           0.        ],
         [-1.38186835, -2.37398046,  0.1039169 , -0.44117647,
           0.        ],
         [-1.39696834, -2.36478489,  0.10558182, -0.44117647,
           0.        ],
         ...,
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ]],
 
        [[-1.41739809, -2.52441286,  0.11184927, -0.44117647,
           0.        ],
         [-1.43217456, -2.51157039,  0.11540743, -0.44901961,
           0.        ],
         [-1.44747723, -2.50033014,  0.1179675 , -0.44509804,
           0.        ],
         ...,
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.

In [14]:
dataset[1]['voxels'].shape

(20801, 100, 5)

In [15]:
from torchsummary import summary

ModuleNotFoundError: No module named 'torchsummary'

In [16]:
net

VoxelNet(
  (voxel_feature_extractor): PillarFeatureNet(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): DefaultArgLayer(in_features=10, out_features=64, bias=False)
        (norm): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (middle_feature_extractor): PointPillarsScatter()
  (rpn): RPNV2(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        (1): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [13]:
print(net)

VoxelNet(
  (voxel_feature_extractor): PillarFeatureNet(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): DefaultArgLayer(in_features=10, out_features=64, bias=False)
        (norm): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (middle_feature_extractor): PointPillarsScatter()
  (rpn): RPNV2(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        (1): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [9]:
# # [line for line in dataset[0]['reg_targets'] for i in line if i != 0]
# res = []
# for line in dataset[1]['reg_targets']:
#     for i in line:
#         if i != 0:
#             print(line)
#             res.append(line)
#             break
# #     break
# res

In [10]:
len([i for i in dataset[1]['labels'] if i == 0])

29710

In [11]:
eval_dataset = input_reader_builder.build(
    eval_input_cfg,
    model_cfg,
    training=False,
    voxel_generator=voxel_generator,
    target_assigner=target_assigner)
eval_dataloader = torch.utils.data.DataLoader(
    eval_dataset,
    batch_size=input_cfg.batch_size,
    shuffle=True,
    num_workers=eval_input_cfg.preprocess.num_workers,
    pin_memory=False,
    collate_fn=merge_second_batch)

feature_map_size [1, 100, 150]
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.208 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [12]:
len(eval_dataset)

47

In [13]:
eval_dataset[0]

{'voxels': array([[[-4.08475634e+00, -3.72370805e+00, -2.74309566e-02,
          -4.41176471e-01,  1.49851084e-01],
         [-4.01466480e+00, -3.78245768e+00, -2.70756963e-02,
          -4.41176471e-01,  1.49851084e-01],
         [-4.03812432e+00, -3.76215380e+00, -2.69570769e-02,
          -4.41176471e-01,  1.49851084e-01],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00]],
 
        [[-3.37062366e+00,  4.60670390e+00,  2.23321315e-02,
          -3.98039216e-01,  0.00000000e+00],
         [-3.34374311e+00,  4.63095813e+00,  2.18319744e-02,
          -3.94117647e-01,  0.00000000e+00],
         [-3.30744228e+00,  4.67251033e+00,  6.60362162e-01,
          -3.78431373e-01,  1.99628115e-01],
         ...,
     

In [14]:
model_dir

PosixPath('/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug')

In [15]:

model_dir = model_dir / cur_time
model_dir.mkdir(parents=True, exist_ok=True)

In [16]:
model_dir

PosixPath('/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug/2021-2-13_11:1')

In [17]:

model_logging = SimpleModelLog(model_dir)
model_logging.open()
model_logging.log_text(proto_str + "\n", 0, tag="config")

start_step = net.get_global_step()
total_step = train_cfg.steps
t = time.time()
steps_per_eval = train_cfg.steps_per_eval
clear_metrics_every_epoch = train_cfg.clear_metrics_every_epoch

amp_optimizer.zero_grad()
step_times = []
step = start_step
run = True
ave_valid_loss = 0.0


model: {
  second: {
    network_class_name: "VoxelNet"
    voxel_generator {
      full_empty_part_with_mean: false
      point_cloud_range : [-30, -20, -2.6, 30, 20, 2.2] #[-80, -20, -2 , 60, 40, 3] # will use [0,1,3,4] means [-100,-100,100,1000]
      voxel_size : [0.1, 0.1, 4]
      max_number_of_points_per_voxel : 100
    }
    voxel_feature_extractor: {
      module_class_name: "PillarFeatureNet"
      num_filters: [64]
      with_distance: false
      num_input_features: 5
    }
    middle_feature_extractor: {
      module_class_name: "PointPillarsScatter"
      downsample_factor: 1
      num_input_features: 64
    }
    rpn: {
      module_class_name: "RPNV2"
      layer_nums: [3, 5, 5]
      layer_strides: [1, 2, 2]
      num_filters: [64, 128, 256]
      upsample_strides: [0.25, 0.5, 1]
      num_upsample_filters: [128, 128, 128]
      use_groupnorm: false
      num_groups: 32
      num_input_features: 64
    }
    loss: {
      classification_loss: {
        weighted_sigmoid

In [18]:
# Ver2
print(total_step)
try:
    start_tic = time.time()
    print("num samples: %d" % (len(dataset)))
    while run == True:
        if clear_metrics_every_epoch:
            net.clear_metrics()
        for example in tqdm_notebook(dataloader):
            lr_scheduler.step(net.get_global_step())
            example.pop("metrics")
            example_torch = example_convert_to_torch(example, float_dtype)

            ret_dict = net_parallel(example_torch)
            loss = ret_dict["loss"].mean()
            cls_loss_reduced = ret_dict["cls_loss_reduced"].mean()
            loc_loss_reduced = ret_dict["loc_loss_reduced"].mean()

            if train_cfg.enable_mixed_precision:
                if net.get_global_step() < 100:
                    loss *= 1e-3
                with amp.scale_loss(loss, amp_optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 10.0)
            amp_optimizer.step()
            amp_optimizer.zero_grad()
            net.update_global_step()

            cls_preds = ret_dict["cls_preds"]
            labels = example_torch["labels"]
            cared = ret_dict["cared"]

            net_metrics = net.update_metrics(cls_loss_reduced,
                                             loc_loss_reduced, cls_preds,
                                             labels, cared)
            step_time = (time.time() - t)
            step_times.append(step_time)
            t = time.time()
            metrics = {}
            global_step = net.get_global_step()

            if global_step % display_step == 0:
                net.eval()
                det = net(example_torch)
                net.train()
                eta = time.time() - start_tic
                if measure_time:
                    for name, val in net.get_avg_time_dict().items():
                        print(f"avg {name} time = {val * 1000:.3f} ms")

                metrics["step"] = global_step
                metrics['epoch'] = global_step / len(dataloader)
                metrics['steptime'] = np.mean(step_times)
                metrics['valid'] = ave_valid_loss
                step_times = []

                metrics["loss"] = net_metrics['loss']['cls_loss'] + net_metrics['loss']['loc_loss']
                metrics["cls_loss"] = net_metrics['loss']['cls_loss']
                metrics["loc_loss"] = net_metrics['loss']['loc_loss']

                if model_cfg.use_direction_classifier:
                    dir_loss_reduced = ret_dict["dir_loss_reduced"].mean()
                    metrics["dir_rt"] = float(
                        dir_loss_reduced.detach().cpu().numpy())

                metrics['lr'] = float(amp_optimizer.lr)
                metrics['eta'] = time_to_str(eta)
                model_logging.log_metrics(metrics, global_step)

                net.clear_metrics()
            if global_step % steps_per_eval == 0:
                torchplus.train.save_models(model_dir, [net, amp_optimizer],
                                            net.get_global_step())
                model_logging.log_text(f"Model saved: {model_dir}, {net.get_global_step()}", global_step)
                net.eval()
                result_path_step = result_path / f"step_{net.get_global_step()}"
                result_path_step.mkdir(parents=True, exist_ok=True)
                model_logging.log_text("########################", global_step)
                model_logging.log_text(" EVALUATE", global_step)
                model_logging.log_text("########################", global_step)
                model_logging.log_text("Generating eval predictions...", global_step)
                t = time.time()
                detections = []
                prog_bar = ProgressBar()
                net.clear_timer()
                cnt = 0
                for example in tqdm_notebook(iter(eval_dataloader)):
                    example = example_convert_to_torch(example, float_dtype)
                    detections += net(example)
                sec_per_ex = len(eval_dataset) / (time.time() - t)
                model_logging.log_text(
                    f'generate eval predictions finished({sec_per_ex:.2f}/s). Start eval:',
                    global_step)
                result_dict = eval_dataset.dataset.evaluation(
                    detections, str(result_path_step)
                )
                for k, v in result_dict['results'].items():
                    model_logging.log_text(f"Evaluation {k}", global_step)
                    model_logging.log_text(v, global_step)
                model_logging.log_metrics(result_dict["detail"], global_step)
                with open(result_path_step/"result.pkl", "wb") as f:
                    pickle.dump(detections,f)
                net.train()

            step += 1
            if step >= total_step:
                break
        if step >= total_step:
            break
except Exception as e:
    model_logging.log_text(str(e), step)
    model_logging.log_text(json.dumps(example['metadata'], indent=2), step)
    torchplus.train.save_models(model_dir, [net, amp_optimizer], step)
    raise e
finally:
    model_logging.close()
torchplus.train.save_models(model_dir, [net, amp_optimizer], step)
# python /kaggle/code/ConeDetectionPointpillars/second/data/nusc_eval.py --root_path="/media/starlet/LdTho/data/sets/nuscenes/v1.0-trainval" --version="v1.0-trainval" --eval_version=detection_cvpr_2019 --res_path="/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/results/step_20/result_nusc.json" --eval_set=val --output_dir="/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/results/step_20"

35000
num samples: 3125


<ipython-input-18-f32186848f57>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for example in tqdm_notebook(dataloader):


  0%|          | 0/3125 [00:00<?, ?it/s]

step=100, epoch=0.032, steptime=0.8591, valid=0.0, loss=2.366, cls_loss=0.2524, loc_loss=2.113, lr=0.0001001, eta= 0 hr 01 m
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
step=200, epoch=0.064, steptime=0.8437, valid=0.0, loss=2.7, cls_loss=0.5606, loc_loss=2.139, lr=0.0001004, eta= 0 hr 02 m
step=300, epoch=0.096, steptime=0.8546, valid=0.0, loss=2.389, cls_loss=0.278, loc_loss=2.111, lr=0.000101, eta= 0 hr 04 m
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
step=400, epoch=0.128, steptime=0.859, valid=0.0, loss=2.1, cls_loss=0.2088, loc_loss=1.892, lr=0.0001018, eta= 0 hr 05 m
Gradient overflow.  Skipping st

<ipython-input-18-f32186848f57>:89: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for example in tqdm_notebook(iter(eval_dataloader)):


...

step=34400, epoch=11.01, steptime=0.8782, valid=0.0, loss=2.444, cls_loss=0.2767, loc_loss=2.167, lr=2.03e-06, eta= 8 hr 28 m
step=34500, epoch=11.04, steptime=0.8703, valid=0.0, loss=2.263, cls_loss=0.3537, loc_loss=1.91, lr=1.414e-06, eta= 8 hr 29 m
step=34600, epoch=11.07, steptime=0.8812, valid=0.0, loss=2.262, cls_loss=0.3609, loc_loss=1.901, lr=9.094e-07, eta= 8 hr 31 m
step=34700, epoch=11.1, steptime=0.8642, valid=0.0, loss=2.3, cls_loss=0.3346, loc_loss=1.966, lr=5.168e-07, eta= 8 hr 32 m
step=34800, epoch=11.14, steptime=0.8602, valid=0.0, loss=2.544, cls_loss=0.5464, loc_loss=1.997, lr=2.36e-07, eta= 8 hr 33 m
step=34900, epoch=11.17, steptime=0.8618, valid=0.0, loss=2.017, cls_loss=0.1903, loc_loss=1.827, lr=6.707e-08, eta= 8 hr 35 m
step=35000, epoch=11.2, steptime=0.856, valid=0.0, loss=1.937, cls_loss=0.1848, loc_loss=1.752, lr=1.001e-08, eta= 8 hr 36 m
Model saved: /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug/2021-2-13_11:1, 35000
####

  0%|          | 0/47 [00:00<?, ?it/s]

generate eval predictions finished(4.82/s). Start eval:
python /kaggle/code/ConeDetectionPointpillars/second/data/nusc_eval.py --root_path="/media/starlet/LdTho/data/sets/nuscenes/v1.0-mini" --version="v1.0-mini" --eval_version=detection_cvpr_2019 --res_path="/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug/results/2021-2-13_11:1/step_35000/result_nusc.json" --eval_set=mini_train --output_dir="/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug/results/2021-2-13_11:1/step_35000"


INFO - 2021-02-13 19:39:49,220 - summary - Summary name eval.nusc/traffic_cone/dist@0.5 is illegal; using eval.nusc/traffic_cone/dist_0.5 instead.
INFO - 2021-02-13 19:39:49,221 - summary - Summary name eval.nusc/traffic_cone/dist@1.0 is illegal; using eval.nusc/traffic_cone/dist_1.0 instead.
INFO - 2021-02-13 19:39:49,221 - summary - Summary name eval.nusc/traffic_cone/dist@2.0 is illegal; using eval.nusc/traffic_cone/dist_2.0 instead.
INFO - 2021-02-13 19:39:49,222 - summary - Summary name eval.nusc/traffic_cone/dist@4.0 is illegal; using eval.nusc/traffic_cone/dist_4.0 instead.
WARNING - 2021-02-13 19:39:49,223 - x2num - NaN or Inf found in input tensor.
WARNING - 2021-02-13 19:39:49,224 - x2num - NaN or Inf found in input tensor.
WARNING - 2021-02-13 19:39:49,224 - x2num - NaN or Inf found in input tensor.


Evaluation nusc
Nusc trainval evaluationtraffic_cone Nusc dist AP@0.5, 1.0, 2.0, 4.0 and TP errors
65.60, 67.06, 69.55, 72.99trans_err, scale_err, orient_err, vel_err, attr_err: 0.1215, 0.3302, nan, nan, nan

eval.nusc.traffic_cone.dist@0.5=0.656, eval.nusc.traffic_cone.dist@1.0=0.6706, eval.nusc.traffic_cone.dist@2.0=0.6955, eval.nusc.traffic_cone.dist@4.0=0.7299, eval.nusc.traffic_cone.trans_err=0.1215, eval.nusc.traffic_cone.scale_err=0.3302, eval.nusc.traffic_cone.orient_err=nan, eval.nusc.traffic_cone.vel_err=nan, eval.nusc.traffic_cone.attr_err=nan


In [ ]:
net.train()

## Visualizing

In [19]:
NameMappingInverse = {
    "barrier": "movable_object.barrier",
    "bicycle": "vehicle.bicycle",
    "bus": "vehicle.bus.rigid",
    "car": "vehicle.car",
    "construction_vehicle": "vehicle.construction",
    "motorcycle": "vehicle.motorcycle",
    "pedestrian": "human.pedestrian.adult",
    "traffic_cone": "movable_object.trafficcone",
    "trailer": "vehicle.trailer",
    "truck": "vehicle.truck",
}
def visualize_evaluation(config_path, model_dir, pretrained_path, multi_gpu=False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if isinstance(config_path, str):
        config = pipeline_pb2.TrainEvalPipelineConfig()
        with open(config_path, "r") as f:
            proto_str = f.read()
            text_format.Merge(proto_str, config)
    else:
        config = config_path
        proto_str = text_format.MessageToString(config, indent=2)


    # Read config file
    input_cfg = config.train_input_reader
    eval_input_cfg = config.eval_input_reader
    model_cfg = config.model.second  # model's config
    train_cfg = config.train_config

    # Build neural network
    net = build_network(model_cfg).to(device)

    # Build Model
    target_assigner = net.target_assigner
    voxel_generator = net.voxel_generator
    print("num parameter: ", len(list(net.parameters())))
    torchplus.train.try_restore_latest_checkpoints(model_dir, [net])

    if pretrained_path is not None:
        print('warning pretrain is loaded after restore, careful with resume')
        model_dict = net.state_dict()
        pretrained_dict = torch.load(pretrained_path)

        new_pretrained_dict = {}
        for k, v in pretrained_dict.items():
            if k in model_dict and v.shape == model_dict[k].shape:
                new_pretrained_dict[k] = v
        print("Load pretrained parameters: ")
        for k, v in new_pretrained_dict.items():
            print(k, v.shape)
        model_dict.update(new_pretrained_dict)
        net.load_state_dict(model_dict)
        net.clear_global_step()
        net.clear_metrics()
    if multi_gpu:
        net_parallel = torch.nn.DataParallel(net)
    else:
        net_parallel = net

    optimizer_cfg = train_cfg.optimizer
    loss_scale = train_cfg.loss_scale_factor
    fastai_optimizer = optimizer_builder.build(
        optimizer_cfg,
        net,
        mixed=False,
        loss_scale=loss_scale)
    if loss_scale < 0:
        loss_scale = "dynamic"
    if train_cfg.enable_mixed_precision:
        max_num_voxels = input_cfg.preprocess.max_number_of_voxels * input_cfg.batch_size
        print("max_num_voxels: %d" % (max_num_voxels))

        net, amp_optimizer = amp.initialize(net, fastai_optimizer,
                                            opt_level="O1",
                                            keep_batchnorm_fp32=None,
                                            loss_scale=loss_scale)
        net.metrics_to_float()
    else:
        amp_optimizer = fastai_optimizer
    torchplus.train.try_restore_latest_checkpoints(model_dir, [fastai_optimizer])
    lr_scheduler = lr_scheduler_builder.build(optimizer_cfg, amp_optimizer, train_cfg.steps)
    if train_cfg.enable_mixed_precision:
        float_dtype = torch.float16
    else:
        float_dtype = torch.float32

    if multi_gpu:
        num_gpu = torch.cuda.device_count()
        print(f"MULTI_GPU: use {num_gpu} gpus")
        collate_fn = merge_second_batch_multigpu
    else:
        collate_fn = merge_second_batch
        num_gpu = 1

    eval_dataset = input_reader_builder.build(
        eval_input_cfg,
        model_cfg,
        training=False,
        voxel_generator=voxel_generator,
        target_assigner=target_assigner)
    eval_dataloader = torch.utils.data.DataLoader(
        eval_dataset,
        batch_size=input_cfg.batch_size,
        shuffle=True,
        num_workers=eval_input_cfg.preprocess.num_workers,
        pin_memory=False,
        collate_fn=merge_second_batch)

    # Start visualizing
    net.eval()
    # example = next(iter(dataloader))
    for example in iter(eval_dataloader):
        detection = example_convert_to_torch(example, float_dtype)
        detection = net(detection)
        print(detection)
        filtered_sample_tokens = eval_dataset.dataset.filtered_sample_tokens
        # filtered_sample_tokens = dataset.dataset.filtered_sample_tokens
        index = filtered_sample_tokens.index(detection[0]['metadata']['token'])

        gt_example = eval_dataset.dataset.get_sensor_data(index)
        # gt_example = dataset.dataset.get_sensor_data(index)
        points = gt_example['lidar']['points']
        pc_range = model_cfg.voxel_generator.point_cloud_range
        points = np.array(
            [p for p in points if (pc_range[0] < p[0] < pc_range[3]) & (pc_range[1] < p[1] < pc_range[4]) & (
                    pc_range[2] < p[2] < pc_range[5])])

        gt_boxes = gt_example['lidar']['annotations']['boxes']
        gt_labels = gt_example['lidar']['annotations']['names']
        c = points[:, 3].reshape(-1, 1)
        c = np.concatenate([c, c, c], axis=1)
        points = points[:, 0:3]
        pc = o3d.geometry.PointCloud()
        pc.points = o3d.utility.Vector3dVector(points)
        pc.colors = o3d.utility.Vector3dVector(c)
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0,
                                                                       origin=[-0, -0, -0])
        geo = [pc, mesh_frame]
        geo = add_prediction_per_class(eval_dataset.dataset.nusc,
                                       detection, gt_boxes, gt_labels,
                                       target_assigner.classes, geo)
        o3d.visualization.draw_geometries(geo)

    net.train()


In [20]:
def add_prediction_per_class(nusc, detection, gt_boxes, gt_labels, class_names, geometries):
    color = {
    "traffic_cone": (1,0,0),
    "gt_traffic_cone": (0,1,0),
    "pedestrian": (1,1,0),
    "gt_pedestrian": (0,0,1)
    }
    det_boxes = detection[0]['box3d_lidar'].cpu().detach().numpy()
    det_labels = detection[0]['label_preds'].cpu().detach().numpy()
    det_scores = detection[0]['scores'].cpu().detach().numpy()
    for i, class_name in enumerate(class_names):
        mask = np.logical_and(det_labels == i, det_scores > 0.5)
        class_det_boxes = det_boxes[mask]
        class_det_scores = det_scores[mask]
        class_det_labels = det_labels[mask]
        print(len(class_det_boxes),len(class_det_scores),len(class_det_labels))
        print(class_det_scores)
        class_gt_boxes = gt_boxes[gt_labels == class_name]
        class_gt_labels = gt_labels[gt_labels == class_name]

        rbbox_corners = box_np_ops.center_to_corner_box3d(class_det_boxes[:, :3],
                                                          class_det_boxes[:, 3:6],
                                                          class_det_boxes[:, 6],
                                                          origin=(0.5, 0.5, 0.5), axis=2)
        gt_rbbox_corners = box_np_ops.center_to_corner_box3d(class_gt_boxes[:, :3],
                                                             class_gt_boxes[:, 3:6],
                                                             class_gt_boxes[:, 6],
                                                             origin=(0.5, 0.5, 0.5), axis=2)
        for j in range(len(rbbox_corners)):
            geometries.append(buildBBox(rbbox_corners[j],
                                        color=color[class_name]))
        for j in range(len(gt_rbbox_corners)):
            geometries.append(buildBBox(gt_rbbox_corners[j], 
                                        color=color[f'gt_{class_name}']))
    return geometries

In [21]:
visualize_evaluation('/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug/pipeline.config',
                     '/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug/2021-2-13_11:1', 
                     '/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug/2021-2-13_11:1/voxelnet-35000.tckpt',
                     )

num parameter:  64
Restoring parameters from /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/cones_pp_full_aug/2021-2-13_11:1/voxelnet-35000.tckpt
warning pretrain is loaded after restore, careful with resume
Load pretrained parameters: 
global_step torch.Size([1])
voxel_feature_extractor.pfn_layers.0.linear.weight torch.Size([64, 10])
voxel_feature_extractor.pfn_layers.0.norm.weight torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.bias torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.running_mean torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.running_var torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.num_batches_tracked torch.Size([])
rpn.blocks.0.1.weight torch.Size([64, 64, 3, 3])
rpn.blocks.0.2.weight torch.Size([64])
rpn.blocks.0.2.bias torch.Size([64])
rpn.blocks.0.2.running_mean torch.Size([64])
rpn.blocks.0.2.running_var torch.Size([64])
rpn.blocks.0.2.num_batches_tracked torch.Size([])
rpn.blocks.0.4.weight torch.Size([

[{'box3d_lidar': tensor([[  6.1208,  -1.8895,   0.3867,   0.3803,   0.3744,   0.7621,   1.5547],
        [ -4.4124,  -2.2139,   0.3418,   0.3745,   0.3694,   0.7538,   1.5684],
        [ 10.6683,  -1.6383,   0.3652,   0.3651,   0.3694,   0.7392,   1.5586],
        [  0.3136,  -9.7616,   0.2852,   0.3721,   0.3693,   0.7372,   1.4551],
        [ 15.5641,  -1.2893,   0.3477,   0.3673,   0.3672,   0.7344,   1.5664],
        [  9.7915,  -9.3202,   0.3008,   0.3726,   0.3750,   0.7304,   1.5234],
        [ 13.7291,  -9.2069,   0.3262,   0.3745,   0.3745,   0.7673,   1.5566],
        [ -9.0114, -10.1489,   0.2930,   0.3604,   0.3574,   0.7313,   1.5840],
        [  5.2322,  -9.5119,   0.2285,   0.3542,   0.3547,   0.7274,   1.5439],
        [ -4.2340, -10.0254,   0.1562,   0.3125,   0.3163,   0.6949,   4.2969],
        [ 28.7298,  -8.3021,   0.2285,   0.3719,   0.3763,   0.7813,   1.5645],
        [  1.2005,  -1.9995,   0.4199,   0.3666,   0.3677,   0.7571,   1.5586],
        [ 25.8329,  -8.

[{'box3d_lidar': tensor([[ 12.5595,   1.0988,   0.4043,   0.3579,   0.3618,   0.7555,   1.5273],
        [ 17.3364,  -5.0157,   0.3008,   0.3489,   0.3463,   0.7696,   3.1738],
        [  1.6272, -11.3580,   0.2852,   0.3620,   0.3628,   0.8055,   1.5469],
        [ 28.2105,   3.8710,   0.4434,   0.4074,   0.4031,   1.0491,   1.5498],
        [ 20.8923,  -2.0804,   0.5254,   0.3688,   0.3726,   0.8713,   1.5605],
        [ 24.6955,   0.9975,   0.5234,   0.3620,   0.3627,   0.8418,   1.5410],
        [ 11.0719,  -7.6134,   0.3066,   0.3645,   0.3669,   0.8232,   1.5293],
        [ 24.2990,  10.6541,   0.4023,   0.4068,   0.4040,   1.1679,   1.5605],
        [ 11.2355,  -9.7620,   0.5273,   0.3857,   0.3816,   1.2668,   1.5713],
        [-29.9349, -15.7849,   0.4844,   0.4098,   0.4049,   1.1441,   1.5137],
        [ 26.3067,  18.9868,   0.4102,   0.4023,   0.3985,   1.2077,   1.5508],
        [ 23.8404,  -2.4403,   0.4648,   0.3731,   0.3913,   0.7653,   2.4141],
        [ -2.7690,   9.

[{'box3d_lidar': tensor([[ 14.5459,   2.5271,   0.5957,   0.3528,   0.3564,   0.7586,   1.5293],
        [  5.6364, -12.0405,   0.2715,   0.3429,   0.3450,   0.7483,   1.5645],
        [ 25.1960,  13.2791,   0.4336,   0.4204,   0.4159,   1.2499,   1.5391],
        [ 26.5724,   3.8569,   0.4395,   0.4119,   0.4094,   1.2291,   1.5469],
        [-16.3280,  -8.8766,   0.4082,   0.3596,   0.3598,   0.7551,   1.6514],
        [  5.0598, -11.0858,   0.2637,   0.3470,   0.3467,   0.7304,   2.2969],
        [ 23.1887,   0.2499,   0.4766,   0.4053,   0.4047,   1.1171,   1.5605],
        [-28.6013,   3.7633,   0.8809,   0.3822,   0.3851,   0.9210,   1.5840],
        [  4.3330,  -8.4292,   0.3691,   0.3565,   0.3626,   0.7118,   1.5430],
        [-26.3686, -18.4631,   0.3809,   0.4591,   0.4596,   1.2535,   1.4863],
        [  4.1019, -11.2287,   0.3008,   0.3462,   0.3380,   0.7933,   0.7148],
        [ 14.3841,  11.7929,   0.3809,   0.3985,   0.3934,   1.2334,   1.5293],
        [-18.7120, -17.

[{'box3d_lidar': tensor([[  8.3693,  -1.0462,   0.3652,   0.3736,   0.3730,   0.7313,   1.5449],
        [ 15.8340,  -5.9665,   0.4180,   0.3763,   0.3814,   0.7923,   1.5254],
        [ 13.4255,   1.4275,   0.4336,   0.3738,   0.3718,   0.7732,   1.5469],
        [ 20.2455,  -3.6728,   0.4922,   0.3806,   0.3792,   0.7852,   1.5723],
        [ 11.8371,  -8.0417,   0.3379,   0.3282,   0.3343,   0.7112,   1.5137],
        [  7.5057, -10.1523,   0.3223,   0.3677,   0.3677,   0.7549,   1.5439],
        [ 24.3270,  -1.6745,   0.4766,   0.3533,   0.3559,   0.7713,   1.5391],
        [-29.2969, -13.6981,   0.5938,   0.4014,   0.3938,   1.2749,   1.5273],
        [ 29.4890,   4.9054,   0.4668,   0.4102,   0.4067,   1.1068,   1.5215],
        [ -6.6828,  -3.8211,   0.6426,   0.4045,   0.3996,   1.2884,   1.5801],
        [-14.3144,  -3.4144,   0.6992,   0.3843,   0.3832,   1.1903,   1.5479],
        [ 27.7962,   0.1507,   0.3906,   0.4003,   0.3975,   1.0469,   1.5566],
        [-29.9172, -15.

...

20 20 20
[0.83999664 0.8394709  0.83828294 0.7985012  0.7895584  0.7774689
 0.7727022  0.7711549  0.7709825  0.7696001  0.755637   0.74927074
 0.74761593 0.7409249  0.7343101  0.7241881  0.66050094 0.6599533
 0.60591567 0.5393472 ]
[{'box3d_lidar': tensor([[-2.6412e+01, -1.8678e+01,  3.7500e-01,  3.8997e-01,  3.8744e-01,
          1.2782e+00,  1.5527e+00],
        [-2.9409e+01, -1.2531e+01,  4.9805e-01,  4.1607e-01,  4.1469e-01,
          1.1695e+00,  1.5527e+00],
        [-2.9731e+01, -1.7368e+01,  4.5312e-01,  4.1569e-01,  4.1279e-01,
          1.2597e+00,  1.5947e+00],
        [ 1.2325e+01, -1.3169e+00,  5.6836e-01,  3.5232e-01,  3.5350e-01,
          7.8019e-01,  1.3574e+00],
        [ 1.0669e+01,  7.5607e+00,  8.9453e-01,  3.5196e-01,  3.6412e-01,
          1.0509e+00,  1.5420e+00],
        [ 1.6995e+01, -5.5338e+00,  5.0586e-01,  4.0050e-01,  4.0234e-01,
          7.8096e-01,  1.5332e+00],
        [-1.9682e+01, -5.2935e+00,  4.1406e-01,  3.7536e-01,  3.7648e-01,
          7.6398e

[{'box3d_lidar': tensor([[ 11.6211,   4.1528,   0.3477,   0.3898,   0.3904,   0.7579,   1.5752],
        [  2.2070,   3.3539,   0.3320,   0.3833,   0.3800,   0.7367,   1.5547],
        [  7.3554,   3.6653,   0.3418,   0.3785,   0.3763,   0.7553,   1.5312],
        [ 14.0061,   5.9598,   0.3496,   0.3569,   0.3579,   0.7465,   1.5645],
        [ -7.0453,   5.7645,   0.4297,   0.3633,   0.3593,   0.7443,   1.5645],
        [ 13.9610,   8.2151,   0.4570,   0.3405,   0.3452,   0.7228,   1.4688],
        [ 13.7064,   8.8642,   0.4707,   0.3505,   0.3543,   0.7683,   1.5195],
        [ -1.2660,   3.3694,   0.3281,   0.3698,   0.3687,   0.7399,   1.5674],
        [-16.7160,   8.3483,   0.6191,   0.3409,   0.3459,   0.7119,   1.5527],
        [-16.7583,   9.6401,   0.6621,   0.3559,   0.3597,   0.7850,   1.5869],
        [-17.4298,   7.7759,   0.6465,   0.3647,   0.3701,   0.9730,   1.5381],
        [-10.3850,   8.0948,   0.5488,   0.3717,   0.3748,   0.7952,   1.5674],
        [ -0.8692,  14.

[{'box3d_lidar': tensor([[  8.4702,   7.7585,   0.5742,   0.3476,   0.3490,   0.7354,   1.5684],
        [  8.5283,   8.4570,   0.5645,   0.3358,   0.3371,   0.7276,   1.5283],
        [  8.4941,   9.2488,   0.6328,   0.3229,   0.3264,   0.7269,   1.5137],
        [-16.1324, -11.3174,   0.4785,   0.3672,   0.3601,   0.8811,   1.5605],
        [ 14.8728,   7.8471,   0.4941,   0.3629,   0.3656,   0.7498,   1.5664],
        [ 18.1839,   5.6395,   0.3438,   0.3678,   0.3712,   0.6921,   1.5488],
        [ 27.5937,   3.2066,   0.4785,   0.3891,   0.3862,   1.0527,   1.5410],
        [-18.7192, -11.3826,   0.3633,   0.3888,   0.3850,   1.0977,   1.5449],
        [ 15.0454,   8.5556,   0.5176,   0.3357,   0.3409,   0.7062,   1.5400],
        [ 24.0088,   3.2062,   0.3672,   0.3675,   0.3647,   0.8320,   0.0730],
        [-10.9411, -11.9176,   0.5762,   0.3887,   0.3892,   1.2710,   1.5449],
        [ 17.9783,   6.5629,   0.3750,   0.3529,   0.3620,   0.7040,   1.6367],
        [-23.4620,   9.

[{'box3d_lidar': tensor([[ -8.0457,  -2.1282,   0.3359,   0.3929,   0.3903,   0.7608,   1.5791],
        [  7.0089,  -1.7149,   0.4043,   0.3767,   0.3749,   0.7560,   1.5449],
        [ -2.4126,  -2.0088,   0.2930,   0.3804,   0.3779,   0.7367,   1.5039],
        [ -3.3972,  -9.7658,   0.3125,   0.3877,   0.3825,   0.7616,   1.5020],
        [ 11.9557,  -1.4410,   0.4297,   0.3625,   0.3677,   0.7614,   1.5312],
        [ 13.3255,  -9.1417,   0.3828,   0.3570,   0.3565,   0.7933,   1.5547],
        [ 10.0220,  -9.2970,   0.3438,   0.3585,   0.3596,   0.7514,   1.5371],
        [ 17.9619,  -1.2619,   0.4102,   0.3690,   0.3683,   0.7547,   1.5215],
        [  1.5616,  -9.5131,   0.3281,   0.3630,   0.3667,   0.7374,   1.5059],
        [ 22.1507,  -8.6798,   0.3496,   0.3661,   0.3645,   0.7823,   1.4961],
        [ 26.4049,  -8.5205,   0.3984,   0.3492,   0.3467,   0.8388,   1.5586],
        [ 25.1063,  -8.6111,   0.4180,   0.3300,   0.3244,   0.7857,   1.5361],
        [  2.5514,  -1.

Process Process-189:
Process Process-192:
Process Process-190:
Process Process-191:
Traceback (most recent call last):
